In [1]:
from IPython.display import display, HTML 
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
 """))

<span style="color:red; font-size: 35px; font-weight: bold;"> ch14.웹데이터 수집 </span></br>
# 1절. BeautifulSoup과 parser

pip install bs4 아나콘다를 설치하면 7500개여개의 패키지 설치

- 공식 사이트 : https://www.crummy.com/software/BeautifulSoup/
- Docs : https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [1]:
import requests #HTTP 요청 처리 lib
from requests_file import FileAdapter

In [15]:
s = requests.Session() #HTTP 요청관리 위한 세션 객체
s.mount("file://", FileAdapter())
response = s.get('file:///ai_x/lecNote/01_python/data/ch14_sample.html')
response

<Response [200]>

In [16]:
response.status_code #상태 코드  
#200 : 정상 상태
#404 : 없는 페이지
#406 : get,post 오류
#500 : 파이썬 문법 오류 (내가 만든 사이트)

200

In [17]:
response.content #바이너리 형식의 내용

b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90 \xec\x95\x88\xec\x9d\x98 \xeb\x82\xb4\xec\x9a\xa9</div>\r\n  <p>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\xb3\xec\x97\x90\xec\x84\x9c \xed\x99\x9c\xec\x9a\xa9\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4</p>\r\n  <div class="contents">\r\n    \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\xa5\xbc \xec\x96\xb4\xeb\x96\xbb\xea\xb2\x8c \xec\x9e\x91\xec\x84\xb1\xed\x95\x98\xeb\x8a\x90\xeb\x83\x90\xec\x97\x90 \xeb\x94\xb0\xeb\x9d\xbc\r\n    <span>\xeb\x8b\xa4\xeb\xa5\xb8<b>\xec\x9a\x94\xec\x86\x8c\xea\xb0\x80 \xeb\xb0\x98\xed\x99\x98</b></span>\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4\r\n  </div>\r\n  <div>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\

In [6]:
response.content.decode('utf-8')

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [7]:
response.text

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [18]:
#html 파싱
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, #response.text 
                    "html.parser") #soup객체 만들기
# soup 웨크롤링 할 수 있게 보여줌

In [25]:
# 1.soup.select_one('선택자') : 해당 선택자 처음 하나만
el = soup.select_one('h1') #처음 나오는 h1태그 하나만
print('el :', el)
print('el.text :', el.text)
print('el.string :', el.string)
print('el의 속성추가 :', el.attrs) #딕셔너리로 나타냄
print('el의 title속성 :', el.attrs['title']) #title의 속성만 가져오기
print('el의 title속성 :', el.attrs.get('title')) #get함수가 오류가 안 나서 좋음
print('el의 이름 :', el.name)

el : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
el.text : Hello, CSS
el.string : Hello, CSS
el의 속성추가 : {'class': ['greeting', 'css'], 'id': 'text', 'title': 'greeting'}
el의 title속성 : greeting
el의 title속성 : greeting
el의 이름 : h1


In [33]:
# 2.soup.select('선택자') : 해당 선택자 모든 엘리먼트를 리스트로 나타냄
el = soup.select('h1')
print('리스트 el :', el)
print('el의 text들 :' ,[e.text for e in el])
print('el의 속성들 :', [e.attrs for e in el])
print('el의 class 속성들 :', [e.attrs.get('class') for e in el])

리스트 el : [<h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
el의 text들 : ['Hello, CSS', 'Hi, CSS']
el의 속성들 : [{'class': ['greeting', 'css'], 'id': 'text', 'title': 'greeting'}, {'class': ['css']}]
el의 class 속성들 : [['greeting', 'css'], ['css']]


In [41]:
#select_one('속성자')와 find(태그,속성) 비교
print('select_one :',soup.select_one('h1.css'))
print('find :', soup.find('h1',{'class':'css'}))
print('find :', soup.find('h1', class_='greeting'))
print()
print('select_one :', soup.select_one('h1#text'))
print('find :', soup.find('h1', {'id':'text'}))
print()
print('select')

select_one : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
find : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
find : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>

select_one : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
find : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>

select


In [44]:
# select('선택자')와 find_all(태그,속성) 비교
print('모든 h1.css, span 태그(select) :', 
      soup.select('h1.css, span'))
print('모든 h1.css, span 태그(find_all) :',
     soup.find_all(['h1','span'],class_='css')) #span은 안 들어온거 아닌지?
print() #개행

모든 h1.css, span 태그(select) : [<h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]
모든 h1.css, span 태그(find_all) : [<h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]


In [49]:
# 없는 엘리먼트 찾기  (a태그)
print('find_all :', soup.find_all('a', class_='css')) #없으면 빈리스트로
print('find :', soup.find('a',class_='css')) #없으면 none으로
print('select :', soup.select('a.css')) #빈 리스트로
print('select_one :', soup.select_one('a.css')) #없으면 none으로

find_all : []
find : None
select : []
select_one : None


# 2절. 정적 웹 데이터 수집(정적 웹크롤링)
- json,html, xml
## 2.1 JSON파일
- request모둘(get)
- urllib.request모듈(urlopen)

In [ ]:
# 크롤링 허용 범위는 사이트마다 다름 / robots.txt에서 확인할 수 있습니다.

In [133]:
# 방법 1. json
import requests
response = requests.get('http://api.github.com')
response, response.status_code

(<Response [403]>, 403)

In [132]:
# 방법 2. json
from urllib.request import urlopen
response = urlopen('http://api.gitgub.com')
response

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [135]:
text = '{"속성1":"값1", "속성2":"값2"}'
type(text)

str

In [136]:
# 문자(딕셔너리 타입)를 딕셔너리
# '{"속성1":"값1", "속성2":"값2"}' => {"속성1":"값1", "속성2":"값2"}
import json 
json.loads(text)

{'속성1': '값1', '속성2': '값2'}

## 2.2html 파일
### 1)환율정보 가져오기(네이버->증권->시장지표)
https://finance.naver.com/marketindex/

In [76]:
# 방법 1. requests
import requests 
from bs4 import BeautifulSoup
url = 'https://finance.naver.com/marketindex/'
response = requests.get(url)
# response, response.status_code
# response.content
# response.content.decode('cp949') == response.text
soup = BeautifulSoup(response.text,
                    'html.parser')

In [81]:
# 방법 2. urlopen
from urllib.request import urlopen
response = urlopen(url)
# response.status 상태 코드 (정상상태 확인)
# response.read() #content와 동일한 값 출력
# response.read().decode('cp949')
soup = BeautifulSoup(response,'html.parser') #text 사용 불가 

In [99]:
title = soup.select('h3.h_lst > span.blind')
for t in title:
    print(t.text, end='\n')

미국 USD
일본 JPY(100엔)
유럽연합 EUR
중국 CNY
달러/일본 엔
유로/달러
영국 파운드/달러
달러인덱스
WTI
휘발유
국제 금
국내 금


In [107]:
price = soup.select('div.head_info > span.value')
[p.text for p in price]
[round(float(p.text.replace(',','')))for p in price]
[round(float(''.join(p.text.split(',')))) for p in price]

[1375, 953, 1555, 191, 144, 1, 1, 99, 61, 1633, 3300, 146173]

In [96]:
#,기준으로 빈스트링으로 연결 join
out = '1,374.70'
round(float(''.join(out.split(','))))

1375

In [113]:
unit = soup.select('div.head_info > span > span.blind')
unit = [u.text for u in unit]
unit.insert(7,'') #7번째 index에 '' 추가
unit

['원', '원', '원', '원', '엔', '달러', '달러', '', '달러', '원', '달러', '원']

In [115]:
status = soup.select('div.head_info > span.blind')
[s.text for s in status]
# status

['하락', '하락', '하락', '하락', '상승', '하락', '하락', '상승', '하락', '하락', '하락', '상승']

In [116]:
[t.text for t in title]
[p.text for p in price]
unit
[s.string for s in status]

['하락', '하락', '하락', '하락', '상승', '하락', '하락', '상승', '하락', '하락', '하락', '상승']

In [117]:
len(title), len(price), len(unit), len(status)

(12, 12, 12, 12)

In [123]:
for idx in range(len(title)):
    print("{}.{} : {}{}-{}".format(idx+1,
                                    title[idx].text,
                                    price[idx].text,
                                    unit[idx],
                                    status[idx].text))

1.미국 USD : 1,375.00원-하락
2.일본 JPY(100엔) : 952.51원-하락
3.유럽연합 EUR : 1,555.06원-하락
4.중국 CNY : 191.06원-하락
5.달러/일본 엔 : 144.3800엔-상승
6.유로/달러 : 1.1342달러-하락
7.영국 파운드/달러 : 1.3520달러-하락
8.달러인덱스 : 99.4200-상승
9.WTI : 60.89달러-하락
10.휘발유 : 1633.17원-하락
11.국제 금 : 3300.4달러-하락
12.국내 금 : 146173.4원-상승


In [126]:
for t,p,u,s in zip(title,price,unit,status):
    print("{} : {}{}-{}".format(t.text,
                                p.text,
                                u,
                                s.text))

미국 USD : 1,375.00원-하락
일본 JPY(100엔) : 952.51원-하락
유럽연합 EUR : 1,555.06원-하락
중국 CNY : 191.06원-하락
달러/일본 엔 : 144.3800엔-상승
유로/달러 : 1.1342달러-하락
영국 파운드/달러 : 1.3520달러-하락
달러인덱스 : 99.4200-상승
WTI : 60.89달러-하락
휘발유 : 1633.17원-하락
국제 금 : 3300.4달러-하락
국내 금 : 146173.4원-상승


In [128]:
for idx,(t,p,u,s) in enumerate(zip(title,price,unit,status)):
    print("{}.{} : {}{}-{}".format(idx+1,
                                t.text,
                                p.text,
                                u,
                                s.text))

1.미국 USD : 1,375.00원-하락
2.일본 JPY(100엔) : 952.51원-하락
3.유럽연합 EUR : 1,555.06원-하락
4.중국 CNY : 191.06원-하락
5.달러/일본 엔 : 144.3800엔-상승
6.유로/달러 : 1.1342달러-하락
7.영국 파운드/달러 : 1.3520달러-하락
8.달러인덱스 : 99.4200-상승
9.WTI : 60.89달러-하락
10.휘발유 : 1633.17원-하락
11.국제 금 : 3300.4달러-하락
12.국내 금 : 146173.4원-상승


### 2) 이번주 로또번호 출력 
- google에 "로또번호 당첨번호" 검색
https://dhlottery.co.kr/gameResult.do?method=byWin


In [140]:
#방법 1의 soup객체 생성
import requests
from bs4 import BeautifulSoup
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')
# response.status_code

200

```
1173회 (2025년 05년 24일 추첨)
당첨번호 [1 5 18 20 30 35]
보너스 3
```

In [261]:
time = soup.select_one('div.win_result > h4 > strong').text
date = soup.select_one('div.win_result > p.desc').text
title1 = soup.select_one('div.num.win > strong').text
win = soup.select('div.num.win > p> span')
winnum = [int(w.text) for w in win]
bonusnum = soup.select_one('div.num.bonus > p > span').text
title2 = soup.select_one('div.num.bonus > strong').text
print(time,date)
print(title1, ":", winnum)
print(title2, ":", [int(bonusnum)])

1173회 (2025년 05월 24일 추첨)
당첨번호 : [1, 5, 18, 20, 30, 35]
보너스 : [3]


In [210]:
num = soup.select('div.win_result > h4 > strong')
[n.text for n in num]
# num

['1173회']

In [198]:
date = soup.select('div.win_result > p')
[d.text for d in date]

['(2025년 05월 24일 추첨)']

In [199]:
title = soup.select('div.num.win > strong')
[t.text for t in title]

['당첨번호']

In [205]:
winnum = soup.select('div.num.win > P')
[w.text for w in winnum]

['\n1\n5\n18\n20\n30\n35\n']

In [206]:
bonusnum = soup.select('div.num.bonus > p')
[b.text for b in bonusnum]

['3']

In [256]:
#방법 2의 soup객체 생성
from urllib.request import urlopen
response = urlopen(url)
soup = BeautifulSoup(response,'html.parser')

In [302]:
win_result = soup.find('div', class_='win_result')
times = win_result.find('strong').text
date = soup.find('p', class_='desc').text
print(times, date)
num_win = soup.find('div',class_=['win'])
title1 = num_win.find('strong').text
lotto_number = num_win.find_all('span') #배열
print(title1,[int(lotto.text) for lotto in lotto_number])
title2 = num_bonus.find('strong').text
num_bonus = soup.find('div', class_=['bonus'])
bonus_num = num_bonus.find_all('span')
print(title2,[int(bonus.text) for bonus in bonus_num])

1173회 (2025년 05월 24일 추첨)
당첨번호 [1, 5, 18, 20, 30, 35]
보너스 [3]


### 3) 다음 검색 리스트
https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&q=%EC%95%BC%EA%B5%AC+
```
no    title    link
0    [비즈 나우] 비트코인               https://v.daum.net/v/20250528075215864
1    [비트코인 2025] 백악관 크립토 차르  https://v.daum.net/v/20250528103907230
```

In [321]:
keyword = '비트코인'
url = 'https://search.daum.net/search?nil_suggest=btn&w=news&DA=SBC&cluster=y&q='\
        +keyword
print(url)

https://search.daum.net/search?nil_suggest=btn&w=news&DA=SBC&cluster=y&q=비트코인


In [333]:
# 웹크롤링 방법1 (리스트안에 딕셔너리로 만들기)
import requests
from bs4 import BeautifulSoup
keyword = '비트코인'
url = 'https://search.daum.net/search?nil_suggest=btn&w=news&DA=SBC&cluster=y&q='\
        +keyword
response = requests.get(url)
soup = BeautifulSoup(response.text,
                    'html.parser')
items_find_list=[] #검색할 결과를 담을 dict list
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
len(items_el)
for idx, item in enumerate(items_el):
    items_find_list.append({'no': idx,
                            'title' : item.text,
                            'link' : item.attrs.get('href')})
import pandas as pd
pd.DataFrame(items_find_list)

,no,title,link
0,0,[비즈 나우] 비트코인 2025 컨퍼런스 개막…'전략자산' 선언 코앞,http://v.daum.net/v/20250528075215864
1,1,[비트코인 2025] 백악관 크립토 차르 “美 정부 비트코인 추가 매입 검토…부채...,http://v.daum.net/v/20250528103907230
2,2,"비트코인, 트럼프 미디어 비축 소식에도 주춤…1억5100만원대",http://v.daum.net/v/20250528095112897
3,3,“맥O날드보다 맛없어!” 혹평이 가득한 트럼프 만찬과 비트코인 피자데이[엠블록레터],http://v.daum.net/v/20250528143002735
4,4,"美 상원의원 ""트럼프 대통령, 비트코인법 지지""",http://v.daum.net/v/20250528090342853
5,5,[비트코인 2025] 로빈후드 창업자 “토큰화 증권은 美 ‘자본 패권’ 키우는 수단”,http://v.daum.net/v/20250528110600761
6,6,"'비트코인 빚투' 스트레티지, 또 샀다…보유량 58만개 돌파",http://v.daum.net/v/20250528042404292
7,7,"""비트코인 산다""…트럼프家, 25억달러 자금 조달 추진[코인브리핑]",http://v.daum.net/v/20250528111837527
8,8,"숨 고르는 비트코인, 10만8000달러선 '주춤'",http://v.daum.net/v/20250528095921258
9,9,"트럼프미디어그룹, 25억 달러 규모 자금 조달 통해 비트코인 매입 예고",http://v.daum.net/v/20250528084245020


In [344]:
# 리스트안에 리스트 만들기
import requests
from bs4 import BeautifulSoup
keyword = '비트코인'
url = 'https://search.daum.net/search?nil_suggest=btn&w=news&DA=SBC&cluster=y&q='\
        +keyword
response = requests.get(url)
soup = BeautifulSoup(response.text,
                    'html.parser')
items_find_list=[] #검색한 결과를 담을 2차원 리스트
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
for idx, item in enumerate(items_el):
    items_find_list.append([idx, item.text, item.attrs.get('href')])

# items_find_list
import pandas as pd
pd.DataFrame(items_find_list, columns=['no','title','link'])

,no,title,link
0,0,[비트코인 2025] 백악관 크립토 차르 “美 정부 비트코인 추가 매입 검토…부채...,http://v.daum.net/v/20250528103907230
1,1,[비즈 나우] 비트코인 2025 컨퍼런스 개막…'전략자산' 선언 코앞,http://v.daum.net/v/20250528075215864
2,2,“맥O날드보다 맛없어!” 혹평이 가득한 트럼프 만찬과 비트코인 피자데이[엠블록레터],http://v.daum.net/v/20250528143002735
3,3,"블랙록, 자사 비트코인 ETF 보유량 25% 확대…기관 투자 본격화 신호탄",http://v.daum.net/v/20250528151802080
4,4,"비트코인, 트럼프 미디어 비축 소식에도 주춤…1억5100만원대",http://v.daum.net/v/20250528095112897
5,5,"'비트코인 빚투' 스트레티지, 또 샀다…보유량 58만개 돌파",http://v.daum.net/v/20250528042404292
6,6,"美 상원의원 ""트럼프 대통령, 비트코인법 지지""",http://v.daum.net/v/20250528090342853
7,7,"숨 고르는 비트코인, 10만8000달러선 '주춤'",http://v.daum.net/v/20250528095921258
8,8,[비트코인 2025] 로빈후드 창업자 “토큰화 증권은 美 ‘자본 패권’ 키우는 수단”,http://v.daum.net/v/20250528110600761
9,9,"트럼프미디어그룹, 25억 달러 규모 자금 조달 통해 비트코인 매입 예고",http://v.daum.net/v/20250528084245020


In [358]:
# 다음 뉴스 검색(키워드,원하는 페이지수)
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import time
keyword = '비트코인'
page = 3
# url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&q={keyword}&p={page}'
# response = requests.get(url)

url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y'
params = {'q':keyword, 'p':page}
response = requests.get(url,params=params)
soup = BeautifulSoup(response.text,
                     'html.parser')

items_find_list=[] #검색할 결과를 담을 dict list
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
len(items_el)
for idx, item in enumerate(items_el):
    items_find_list.append({'no': idx,
                            'title' : item.text,
                            'link' : item.attrs.get('href')})
    print({'no':(page-1)*10+ idx,
                            'title' : item.text,
                            'link' : item.attrs.get('href')})

{'no': 20, 'title': ' [영상] “강세장 진입한 비트코인, 내년 2분기까지 최소 20만~30만 달러 갈 듯” ', 'link': 'http://v.daum.net/v/20250527090137566'}
{'no': 21, 'title': ' 전력거래소, `공감·혁신` 토대 조직문화 바꾸기 ', 'link': 'http://v.daum.net/v/20250528152647580'}
{'no': 22, 'title': ' “미국 대통령이 이래도 되나”…트럼프 일가, 4조원 유치해 대놓고 코인 투자 ', 'link': 'http://v.daum.net/v/20250527220300328'}
{'no': 23, 'title': ' 트럼프미디어, 비트코인 3.4조 투자 발표에 주가 10% 폭락 ', 'link': 'http://v.daum.net/v/20250528080206055'}
{'no': 24, 'title': ' 트럼프 미디어, 비트코인 사나…가상자산 매입 위해 30억 달러 조달 추진 ', 'link': 'http://v.daum.net/v/20250527043000368'}
{'no': 25, 'title': ' 전력거래소, 2025년 공감과 혁신 특강 개최 ', 'link': 'http://v.daum.net/v/20250528150120264'}
{'no': 26, 'title': ' <단독>"예배 참석하면 코인 드려요. 5년 안에 비트코인 넘어섭니다"..신앙심 노린 코인 유사 사기 기승 ', 'link': 'http://v.daum.net/v/20250528083323774'}
{'no': 27, 'title': ' 뉴욕증시 휴장에 10.9만弗선 숨고른 비트코인 [매일코인] ', 'link': 'http://v.daum.net/v/20250527103810840'}
{'no': 28, 'title': ' 이재명 공약집…권력기관 고강도 개혁·AI 통한 성장에 방점 ', 'link': 'http://v.daum.net/v/202505281

In [ ]:
# 함수 만들기 

In [361]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import time
def collect_list(keyword,page=1):
    'keyword로 다음 검색한 결과(해당 page)를 return'
    url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y'
    params = {'q':keyword, 'p':page}
    response = requests.get(url,params=params)
    soup = BeautifulSoup(response.text,
                         'html.parser')
    items_find_list=[] #검색할 결과를 담을 dict list
    items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
    for idx, item in enumerate(items_el):
        items_find_list.append({'no':(page-1)*10+ idx,
                                'title' : item.text,
                                'link' : item.attrs.get('href')})
    return items_find_list

In [366]:
collect_list('빨래',4)

[{'no': 30,
  'title': ' 어쩌면 마지막 빨래 ',
  'link': 'http://v.daum.net/v/20250421143123301'},
 {'no': 31,
  'title': ' ‘관세전쟁’에 빨래 횟수도 줄이는 미국인들 ',
  'link': 'http://v.daum.net/v/20250425123746580'},
 {'no': 32,
  'title': ' 밥 짓기·설거지·빨래 강요, 괴롭힘 신고도 묵살···새마을금고 간부들 1700만원 배상 ',
  'link': 'http://v.daum.net/v/20250428152014225'},
 {'no': 33,
  'title': ' 빨래 횟수도 줄이는 미국인들…실적 전망 줄줄이 낮춘다 ',
  'link': 'http://v.daum.net/v/20250425111350870'},
 {'no': 34,
  'title': ' 빨래건조대 혁신… 모터로 건조시간 35%↓ ',
  'link': 'http://v.daum.net/v/20250409012011659'},
 {'no': 35,
  'title': " 문앞에 놔두면 빨래 끝…세탁도 '앱 클릭' ",
  'link': 'http://v.daum.net/v/20250513160655495'},
 {'no': 36,
  'title': ' 박승희, 시모가 곱게 갠 빨래→아수라장 옷방에 철푸덕 (동상이몽2)[결정적장면] ',
  'link': 'http://v.daum.net/v/20250422060825114'},
 {'no': 37,
  'title': ' ‘제이쓴♥’ 홍현희, 60억 압구정 아파트 나와 집 없이 떠돌아다녀 “벌써 10일째” ',
  'link': 'http://v.daum.net/v/20250528145050679'},
 {'no': 38,
  'title': ' 하원미 "하숙집 엄마처럼 빨래, 밥, 청소"…추신수 월봉 70만원 시절에도 살뜰 내조(\'추신수몰래\') ',
  'link': 'http://

In [ ]:
#원하는 keyword로 다음검색 (append X)

for 

### 4)User-agent를 추가하여 크롤링
- urlopen()함수를 사용하면 크롤링이 안 되는 사이트
- User-agent를 추가하여 크롤링(브라우저)

In [340]:
# 웹크롤링 방법2
from urllib.request import urlopen
import urllib.parse

word = '비트코인'
word = urllib.parse.quote(word)
# print(word)
url = 'https://search.daum.net/search?nil_suggest=btn&w=news&DA=SBC&cluster=y&q='+word
response = urlopen(url)
soup = BeautifulSoup(response,
                    'html.parser')

items_find_list=[] #검색할 결과를 담을 dict list
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
# len(items_el)
for idx, item in enumerate(items_el):
    items_find_list.append({'no': idx,
                            'title' : item.text,
                            'link' : item.attrs.get('href')})
import pandas as pd
pd.DataFrame(items_find_list)


""


꼭 User-Agent를 사용하여야 하는 경우 : https://www.melon.com/chart/index.htm

In [31]:
# 방법 1
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
url='https://www.melon.com/chart/'
# melonpage = requests.get(url)
# melonpage.status_code #오류

406

In [29]:
# 방법2.
from urllib.request import urlopen, Request
# import urllib.parse
# melonpage = urlopen(url) 에러남
# melonpage = requests.get(url)
# melonpage.status_code


In [35]:
#User-Agent 추가
headers = {'user-agent':
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36'}
#방법2
request = Request(url,headers=headers)
melonpage = urlopen(request)

#방법1
melonpage = requests.get(url, headers=headers)
soup = BeautifulSoup(melonpage.text, #melonpage.content
                    'html.parser')
# soup

In [84]:
import requests
from bs4 import BeautifulSoup
url='https://www.melon.com/chart/'
# 멜론 순위, 노래제목, 가수 , 좋아요갯수 
# 1위 | 너에게 닿기를 | 10CM | ♥ 85,756 
title = soup.select('div.ellipsis.rank01 > span')
title
artist = soup.select('div.ellipsis.rank02 > span.checkEllipsis')
artist
len([t.text.strip()for t in title])
len([art.text for art in artist])
for idx,(t,art) in enumerate(zip(title,artist)):
    print('{}순위 | 노래 제목:{} | 가수:{}'.format(idx+1,
                                          t.text.strip(),
                                          art.text))
# [t.text for t in title]
# artist = soup.select_one('div.ellipsis.rank02 > span.checkEllipsis').text
# artist

# print(title,artist)

1순위 | 노래 제목:너에게 닿기를 | 가수:10CM
2순위 | 노래 제목:Never Ending Story | 가수:아이유
3순위 | 노래 제목:Drowning | 가수:WOODZ
4순위 | 노래 제목:like JENNIE | 가수:제니 (JENNIE)
5순위 | 노래 제목:모르시나요(PROD.로코베리) | 가수:조째즈
6순위 | 노래 제목:THUNDER | 가수:세븐틴 (SEVENTEEN)
7순위 | 노래 제목:TOO BAD (feat. Anderson .Paak) | 가수:G-DRAGON
8순위 | 노래 제목:네모의 꿈 | 가수:아이유
9순위 | 노래 제목:Whiplash | 가수:aespa
10순위 | 노래 제목:HOME SWEET HOME (feat. 태양, 대성) | 가수:G-DRAGON
11순위 | 노래 제목:나는 반딧불 | 가수:황가람
12순위 | 노래 제목:어제보다 슬픈 오늘 | 가수:우디 (Woody)
13순위 | 노래 제목:오늘만 I LOVE YOU | 가수:BOYNEXTDOOR
14순위 | 노래 제목:REBEL HEART | 가수:IVE (아이브)
15순위 | 노래 제목:HAPPY | 가수:DAY6 (데이식스)
16순위 | 노래 제목:빨간 운동화 | 가수:아이유
17순위 | 노래 제목:Flower | 가수:오반(OVAN)
18순위 | 노래 제목:APT. | 가수:로제 (ROSÉ), Bruno Mars
19순위 | 노래 제목:청춘만화 | 가수:이무진
20순위 | 노래 제목:HOT | 가수:LE SSERAFIM (르세라핌)
21순위 | 노래 제목:10월 4일 | 가수:아이유
22순위 | 노래 제목:한 페이지가 될 수 있게 | 가수:DAY6 (데이식스)
23순위 | 노래 제목:Last Scene (Feat. 원슈타인) | 가수:아이유
24순위 | 노래 제목:HBD | 가수:세븐틴 (SEVENTEEN)
25순위 | 노래 제목:MY LOVE(2025) | 가수:이예은, 아샤트리, 전건호
26순위 | 노래 제목:그날이 오면 | 가수:투모로우바이투게더